Why DIRECT? in this paper, the threats have been simplified to circular boundaries. In real life, the boundaries will be irregular, and will have no explicit differentiable expression avalible.

In [57]:
from scipy.optimize import direct, Bounds
import numpy as np

s# Deriving the cost function.

## Finding an initial cost function

* Consider problem in two dimensions.
* Route consists of start and endpoints with a number of intermediate waypoints.
* The coordinates of the waypoints are the optimization variables.
* We want the distance flown to be minimal, so calculate Euclidian distance of the route, and label it $L$.
* Determine if any of our path lies within a threat, and the length of the path within the threat.

If the route lies within the $i$-th threat for a distance $L_i$ then the cost of the route can be expressed as

$C = L + \displaystyle\sum_{i} \rho _i L_{i}^{p}$

$\rho _i$ is a penalty parameter for the $i$-th threat.

In other words, the cost is the total length of the path, plus the length of each segment within a threat multiplied by the penalty parameter associated with that particular threat.

## Defining "leg" length
We have $n$ waypoints $w_j = (w_{j1}, w_{j2})$ for $j = 1, ..., n$, let the start and end points be $w_0, w_{n+1}$ respectively. We can define the vector $u_j = w_j - w_{j-1}$ which helps us defined the length of any given leg as 

$l_j = \sqrt{(u_{j1}^2 + u_{j2}^2)}$

## Determining threat violations

We  will use linear interpoliation to estimate where leg $j$ intersects with threat $i$. 

Consider $\sigma _{max}$ to be the maximum step size for use in sampling any leg of a route. This means that the number of samplings points along leg $j$ is $K_{max} = \lceil l_j / \sigma _{max} \rceil$.Let the normalized step size be $\delta \lambda = \frac{1}{K_{max}}$ and let $u_k$ be a sampled point in the leg.

$u_k = w_{j-1} + k \delta \lambda (w_j - w_{j-1})$

$k = [0, ..., K_{max}]$.

Suppose we have an indicator function of position, $T_i$, such that $T_i(u_k) \leq 0$ indicates that the sampled position within the leg, $u_k$ is within threat i, and confirms that the position is not within the threat otherwise. Using what we have defined so far can create an algorithm for calculating "in threat" length, denoted $l_{ji}$.

1. Find $l_j$, $l_i$. Set $i_{ji} = 0$  
2. Set $k=0$, $K_{max}=(\delta \lambda)^{-1}$  
3. If $T_i (u_0) \leq 0$ then $\lambda _b = 0$  
4. For $k = 1, ..., K_{max}$   
5.  ---> If $T_i (u_k) \leq 0$ and $T_i (u_{k-1}) > 0$ then  
6.  ------->  set $\kappa = T_i(u_k)/(T_i(u_k)-T_i(u_{k-1})), \lambda _b = (k - \kappa)\delta \lambda$  
7.  ---> If $T_i (u_k) > 0$ and $T_i (u_{k-1}) \leq 0$ then  
8.  ------->  set $\kappa = T_i(u_k)/(T_i(u_k)-T_i(u_{k-1})), \lambda _e = (k - \kappa)\delta \lambda$  
9.  ------->  set $l_{ji} + (\lambda _e - \lambda _b) l_j$  
10.  ---> If $T_i (u_{K_{max}}) \leq 0$ then   
11.  -------> set $l_{ji} = l_{ji} + (1- \lambda _b )l_j$  

Line-by-line explanation:

1. Find the length of the current leg. Initialize "in threat" distance accumulator as 0.
2. Set up a counter at 0, and set $K_{max} = (\delta \lambda)^{-1}$. Recall: $\delta \lambda = 1/K_{max}$. So we're just initializing $K_{max}$ with it's own value.
3. Determine if our starting point is within the threat.
4. For k = 1 -> $K_{max}$ to start checking each point along the leg.
5. Detect if the path passed INTO the threat.
6. Calculate the location of the boundary with interpolation. $\kappa$ is the fractional interpolation value, it represents a number from 0-1 that says where the boundary crossing lies between two samples on the leg. After that, we calculate where the crossing occurs on the entire leg. We know the crossing occured when we took the k-th step. So $k-\kappa$ is the number of steps to where the boundary is, and $\delta \lambda$ is our step size.
7. Detect if the passed passed OUT of the threat
8. Perform the same interpolation as step 6. Set $\lambda _e$ instead which is the exit position.
9. Since we've exited, we can calculate the in-threat distance, $l_{ji}$. $(\lambda _e - \lambda _b)$ represents the normalized distance in the threat zone and multiplying it by $l_j$ converts it to real distance. Add it to the accumulator. 
10. Determine if exit point is inside the threat
11. Do a similar calculation to step 9, add it to the accumulator.
 
## Defining constraints
Two additional features are included to enhance the realism of the simulation. Firstly, you can restrict routes that involve sharp turns, and secondly, you can set a limit on the distance between waypoints.

The angle between successive legs is defined as $\phi _j$. Given by

$\phi _j = cos^{-1}\{\frac{u_j ^T u_{j+1}}{||u_j|| ||u_{j+1}||}\}$

$\phi _{max}$ is the limiting turn angle, and $l_{min}$ is the minimum leg-length. We can define a final extended cost function.

$$
C = \sum_{j=1}^{n+1} ( l_j + \mu (( l_{min} - l_j )_+ )^2 + \sum_{j=1}^m \rho _i l_{ji}^p ) + \sum_{j=1}^n v ((\phi _j - \phi _{max})_+)^2
$$

$\mu$ and $v$ are positive penalty parameters for controling minimum leg length and turn angle.

Parameters for the cost function: $l_{min}, \phi _{max}, \mu, \nu$
Function will need list of waypoints, start and endpoint, and threat list.

In [ ]:
# Cost function
def cost(x, start, end, threats, lmin, phimax, mu, nu, p):
    waypoints = np.vstack([start, x.reshape(-1, 2), end]) # add start and stop points to our stack
    total_cost = 0 # accumulator for cost

    legs = np.diff(waypoints, axis=0) # gets differences of each leg
    leg_lengths = np.linalg.norm(legs, axis=1) # convert differences to magnitudes
    turn_angles = np.zeros(len(legs) - 1.0) # initialize turn angles as list of 0s

    # Calculate the turn angle between each leg
    for j in range(1, len(legs)):
        u_prev = legs[j-1]
        u_curr = legs[j]
        cos_phi = np.dot(u_prev, u_curr) / (np.linalg.norm(u_prev) * np.linalg.norm(u_curr))
        turn_angles[j] = np.arccos(np.clip(cos_phi, -1, 1))
    
    # First component of cost, total path length
    total_cost += np.sum(leg_lengths)

    # Penalty for leg length 
    short_leg_penalty = np.sum(mu * np.maximum(lmin - leg_lengths, 0) ** 2)

    # Threat penalty by in-threat distance
    for i, (threat_centre, threat_radius) in enumerate(threats): # Check each threat
        l_ji = 0.0 # Accumulator for this threat
        for j in range(len(legs)):
            l_ji += in_threat_distance(waypoints[j], waypoints[j+1], threat_centre, threat_radius, sigma_max)
        total_cost += l_ji ** p # p = threat penalty parameter.


    pass

def in_threat_distance(start, end, center, radius, sigma_max):  # Center and radius are components of the threat.
    l_j = np.linalg.norm(end - start) # start and end are vectors.
    l_ji = 0.0
    K_max = np.ceil(l_j / sigma_max)
    delta_lambda = 1 / K_max

    if np.linalg.norm(start - center) <= radius:
        lambda_b = 0
    

    for k in range(1, K_max+1):


    pass

threats = [(np.array([75,  29]),    9   ),
           (np.array([80,  66]),    15  ),
           (np.array([95,  115]),   22.5),
           (np.array([135, 131]),   20  ),
           (np.array([140, 67.5]),  37.5),
           (np.array([197, 125]),   30  ),]

# Approach 1 - Seperate calculation of outward and return legs
Route is from (50, 30) to (167, 107)
We will use two waypoints:
(108 $\pm$ 60, 68 $\pm$ 40), (109 $\pm$ 60, 69 $\pm$ 40)